<p align="center">
  <img src="Graphics/Episode VIII.png" />
</p>

## (0) The Robot Operating System (ROS), Continued

***Classical planning*** is the problem of finding a sequence of actions for achieving a goal from an initial state assuming that actions have *deterministic* effects and that the environment is *fully observable*.

### (0.1) Task Planning

**Formal Problem Definition:** A task planning problem is a tuple $\langle A,s_{0},g \rangle$, where $A$ is a set of
parameterized propositional actions, $s_{0}$ is an initial state of the domain, and $g$, a set of propositions, is the goal condition. Let $a(s)$ denote the result of applying action $a$ on state $s$, obtained by modifying $s$ to make the positive (negative) literals in the effects of $a$ true (false). A solution to a task planning
problem is a sequence of grounded actions $a_{0},...,a_{n}$ such that $s_{0}$ satisfies the preconditions of $a_{0}$, $s_{i+1} = a_{i}(s_{i})$ satisfies $a_{i+1}$’s preconditions for $i=1,...,n−1$ and $s_{n+1}$ satisfies $g$. [3]

Over the years, there have been several languages used to define domains for task planning. The first task planning language was the `STanford Research Institute Problem Solver` ([STRIPS](https://ai.stanford.edu/users/nilsson/OnlinePubs-Nils/PublishedPapers/strips.pdf)) in 1971, made popular by the [Shakey](https://en.wikipedia.org/wiki/Shakey_the_robot) project [1]. Fun Fact: the still widely-used search algorithm A* was developed for Shakey (it was released in 1968!).
<p align="center">
  <img src="Graphics/Shakey.jpg" />
</p>

Since then, several related languages for planning have come up, but one of the most popular today is the `Planning Domain Definition Language` ([PDDL]((https://www.researchgate.net/publication/2278933_PDDL_-_The_Planning_Domain_Definition_Language))) [2]. The first PDDL paper was published by Drew McDermott and his colleagues in 1998 to be used in the International Artificial Intelligence Planning Systems Conference (AIPS-98) competition, though there have been several enhancements and variations tacked on over the years. [4]

### (0.2) Learning Outcomes

In this tutorial, we will cover:
* What is classical planning?
* What is the Planning Domain Definition Language (PDDL)?
* How do we use PDDL to solve deterministic task planning problems?

### (0.3) Running PDDL Solvers

For those using an IDE like Virtual Studio Code, they will often have a downloadable extension for working with PDDL files and solvers that you might find very convenient (I know I do), and I will show you the PDDL extension in VS Code during this tutorial. Otherwise, you can use an online PDDL solver such as [this one](http://editor.planning.domains/) (you'll just need to upload your domain and problem files to the website).

<p align="center">
  <img src="Graphics/Plan.png" />
</p>

## (1) Planning Domain Definition Language (PDDL)

### (1.1) What's the big idea?
The original PDDL paper by Ghallab et. al. [2] explains that "PDDL is intended to express the 'physics' of a *domain*, that is, what *predicates* there are, what *actions* are possible, what the structure of compound actions is, and what the effects of actions are." We can thus use PDDL to describe the domain (i.e. environment) of interest and the actions possible within in it - independent of any tasks we might want to accomplish - and to describe the classical planning problem we'd like to solve (i.e. the goal state we'd like to reach, given some initial state).

The components of a PDDL planning task include:
* **Objects** – Things in the world that interest us (i.e. robots, cars, houses)
* **Predicates** – Properties of objects that we are interested in; can be true or false (think of them as boolean functions that take objects as their input)
* **Initial State** – The state of the world that we start in (a set of predicates that is true initially)
* **Goal State** – Things that we want to be true (a set of predicates that we seek to achieve)
* **Actions** – Ways of changing the state of the world; they can only be done if certain preconditions (predicates) hold, and they have certain effects on the world (they alter the predicates which hold true)

The syntax for PDDL is inspired by the [Lisp](https://en.wikipedia.org/wiki/Lisp_programming_language) programming language, and in general a PDDL task will be defined using two files: the domain file and the problem file. We input these two files into a PDDL solver (which for now we'll treat as a blackbox AI), and from the solver we will obtain either a valid plan (a sequence of actions that brings us from the initial state to the goal state) or we will learn that there is no valid plan.

<p align="center">
  <img src="Graphics/PDDL_Diagram.png" />
</p>

### (1.2) The Domain File

In a PDDL domain file, we will define the objects that can appear in our world, the predicates that can describe properties of those objects, and the actions that we can take in this world. A domain file consists of:
* `(define (domain DOMAINNAME)`
* a `:requirements` definition (use `:adl :typing` by default)
* definitions of `:types` (each parameter has a type)
* definitions of `:predicates`
* definitions of `:actions`
* `)`

In the actual syntax of PDDL, such a file might look as follows:
```
(define (domain XXX)
    (:requirements XXX)  

    (:predicates (XXX))

    (:action XXX
       :parameters (XXX)
       :precondition (XXX) 
       :effect (XXX)
    )
)
```
In order to further explain what each line means and how to use them, it'd be best to start with an example. Imagine we are in a room where there is an apple on a shelf one side of the room, a robot in the middle of the room, and a table on the other side of the room from the shelf. This world looks as follows:
<p align="center">
  <img src="Graphics/simple_worlds.png" />
</p>

We want to command the robot to perform a simple task like “take the apple from the shelf and put it on the table”. How can we represent this world and this task using PDDL?

**Domain Name:** First off, we start by giving our domain a name, which we do in the first line of the domain file as follows: `(define (domain simple)`. Thus, our domain will just be called `simple`.

**Requirements:** Next, we need to define the requirements for our PDDL planner (think of them like libraries that you might import into Python). We will specify two requirements for this problem: `typing` and `adl`. The former allows us to create custom types to which objects must belong (in Python, types include integer, boolean, and so on - here you can define whatever types you want), while the latter is a "super requirement" that adds a few other useful requirements (for a deeper explanation of `adl` and all the other requirements you can request in PDDL, check out the [documentation](https://planning.wiki/ref/pddl/requirements)). The requirements line will then look as follows: `(:requirements :typing :adl)`.

So far, our domain file looks like this:
```
(define (domain simple)
    (:requirements :typing :adl)
)
```
Now it's time to actually describe our world using predicates and actions (and types!).

**Predicates (Part 1):** Like we said before, predicates are boolean functions whose parameters can be any of the objects in our world. We represent parameters using a question mark followed by some character, for example `?s`. Then a predicate, which we'll just call `foo`, could be defined as `(foo ?s)` (such a predicate, which only takes in one parameter, is called *unary predicate*). This is akin to defining a Python variable `s` and function `foo(s)`.

If we didn't require `typing` earlier, then we could use unary predicates to check whether a given object belongs to a certain category. For example, the predicate `(Robot ?r)` is only true if the input object is a robot (this would be specified in the initial state, in the problem file - more on this later). However, since we will almost always require `typing` in our PDDL domains, we can avoid having to use a bunch of predicates to verify the types of objects - instead, we can define the object types from the beginning and specify that a parameter for a predicate (or an action) must be of a certain type.

**Types:** Backtracking a little bit, we will add a whole new line to our domain file, between requirements and predicates, to define our types. The basic type for an object in PDDL is just `object`, and we can define new types as children of the `object` type (and in fact we can define child types of those child types - like inheritance in OOP). For example, in our problem we see that we have the following relevant physical objects: a robot, a apple, a shelf, and a table. The robot and the apple aren't very similar objects, and it'll serve us well to just define their types to be `robot` and `apple` respectively. The shelf and table, though, or both locations where the apple might be. Thus, for both of them we can define a new type called `location` as a child of the `object` type, without needing to specify separate types for each of them. All in all, the PDDL syntax for this will be as follows: `(:types apple location robot - object)`.

**Predicates (Part 2):** Returning to our predicates now that we have our new types defined, we'll now be able to formulate the predicates with more ease. Let's try to think of some properties of the objects in our problem (or yes/no questions which we can ask about them) that will be useful for describing our world. For example, we can use a predicate to describe where the apple is for a given state. We need to formulate it as a boolean function, so we may ask the question "Is the apple on the shelf?", or the question "Is the apple on the table?", or even "Is the robot holding the apple?". We also want to know where the robot is (since it can't be at both the table and the shelf at the same time), so we should also ask questions like "Is the robot at the table?" and "Is the robot at the shelf?".

Since the table and the shelf both belong to the type `location`, we can define the following predicate to determine if the apple is on one of them: `(On ?a - apple ?l - location)`. This predicate is called `On`, and it takes in two parameters (this makes it a *binary predicate*): an `apple`-type object represented by the input variable `?a` and a `location`-type object denoted by `?l` (note: the character used in the parameter name doesn't have to be the first letter of the object type, it's arbitrary - I just do so for convenience). The fact that we force the input parameters of the predicate to be of a certain type (which is called *static typing*) is different from how functions work in *dynamically-typed* languages like Python (i.e. the functions are agnostic to the types of their inputs - the Python interpreter figures out what types the inputs should be on its own), but for those of you familiar with statically-typed languages like C this concept should be familiar. 

To determine whether or not the robot is holding the apple, we can use another binary predicate which we'll call `Holding` and define as follows: `(Holding ?r - robot ?a - apple)`. Note that the order of the parameters is important (or more precisely, it's important that we be consistent with the order of the parameters), we'll see why later. Finally, we can define a binary predicate called `At` which will tell us whether or not the robot as at some location (whether that be the shelf or the table): `(At ?r - robot ?l - location)`. The value of a predicate is either True or False depending on the current state of the world (think of it like an if statement in Python - if {condition} then {boolean}).

Putting it all together again, our domain file is almost done:
 ```
(define (domain simple)
    (:requirements :typing :adl)

    (:types apple location robot - object)

    (:predicates (On ?a - apple ?l - location)(Holding ?r - robot ?a - apple)(At ?r - robot ?l - location))
)
```
Note that the order in which we define the predicates doesn't matter.

**Actions:** Finally, we need to define the actions that are possible in our world. To describe an action, we need to specify which objects it depends on and/or affects, what the preconditions for doing the action are, and what the effects of the action are. For example, say our robot picks up the apple from some location. The relevant parameters are the robot, apple, and location, while the preconditions that should be met are that the apple is on the location, that the robot is at the location, and that the robot is not already holding the apple. The effects of this action should be that the apple is no longer on the location, and that the robot is holding the apple (the robot shouldn't be moving away from the location just yet, so we don't need to touch that). The definition for such an action in PDDL will look as follows:
```
(:action pick
    :parameters (?a - apple ?r - robot ?l - location)
    :precondition (and (On ?a ?l)
                       (not (Holding ?r ?a))
                       (At ?r ?l))
    :effect (and (not (On ?a ?l))
                 (Holding ?r ?a))
)
```
Whoa whoa whoa. "You never said anything about `and` and `not`, what are those?" you might be thinking. Well, these are examples of *logical operators*, which allows us to create *formulas* based on the Boolean values of two or *atoms* and some desired logical relationship between them. For those who don't have any background in logic, I shall elaborate a bit further:
* An atom is the simplest boolean function possible, they can't be decomposed further - our predicates serve as atoms here
* A (propositional) formula is an expression that contains a sequence of atoms and logical operators on those atoms, for example: $(\phi\land\theta)$ is a formula whose value is either True or False, where $\phi$ and $\theta$ are atoms whose values are either True or False, and $\land$ is the AND (or *conjunction*) logical operator (if both atoms are true then the formula is True, otherwise it's False)
* A logical operator is a symbol used to connect two or more expressions such that the value of the compound expression produced depends only on that of the original expressions and on the meaning of the operator, for example: the OR (or *disjunction*) operator $\lor$, the NOT (or *negation*) operator $\neg$

Back to PDDL, the preconditions and effects of actions will be represented as formulas whose atoms are the predicates we defined. In the case of our `Pick` action, the precondition is the formula (`On(apple,location)`$\land\neg$`Holding(robot,apple)`$\land$`At(robot,location)`), while the effect is the formula ($\neg$`On(apple,location)`$\land$`Holding(robot,apple)`).

Now that we have a better of understanding of how propositional logic and actions in PDDL, we can define two more actions. The first one should represent the robot placing the apple on some location, and looks very similar to the `pick` action we already defined:
```
(:action place
    :parameters (?a - apple ?r - robot ?l - location)
    :precondition (and (not (On ?a ?l))
                       (Holding ?r ?a)
                       (At ?r ?l))
    :effect (and (On ?a ?l)
                 (not (Holding ?r ?a)))
)
```
The second action should describe how the robot actually moves between the shelf and the table:
```
(:action move
    :parameters (?r - robot ?from ?to - location)
    :precondition (and (At ?r ?from)
                       (not (At ?r ?to)))
    :effect (and (At ?r ?to)
                 (not (At ?r ?from)))
)
```
Finally, we can put everything together and obtain our complete domain file:
 ```
(define (domain simple)
    (:requirements :typing :adl)

    (:types apple location robot - object)

    (:predicates (On ?a - apple ?l - location)(Holding ?r - robot ?a - apple)(At ?r - robot ?l - location))

    (:action pick
    :parameters (?a - apple ?r - robot ?l - location)
    :precondition (and (On ?a ?l)
                       (not (Holding ?r ?a))
                       (At ?r ?l))
    :effect (and (not (On ?a ?l))
                 (Holding ?r ?a))
    )

    (:action place
    :parameters (?a - apple ?r - robot ?l - location)
    :precondition (and (not (On ?a ?l))
                       (Holding ?r ?a)
                       (At ?r ?l))
    :effect (and (On ?a ?l)
                 (not (Holding ?r ?a)))
    )

    (:action move
    :parameters (?r - robot ?from ?to - location)
    :precondition (and (At ?r ?from)
                       (not (At ?r ?to)))
    :effect (and (At ?r ?to)
                 (not (At ?r ?from)))
    )   
)
```
You can find this domain as a `.pddl` file called `domain_simple.pddl` in the `Simple Example` folder.

### (1.3) The Problem File

In a PDDL problem file, we instantiate the objects (similar to OOP) in our world, and declare the initial state and goal state for our problem. A problem file consists of:
* `(define (problem PROBLEMNAME)`
* declaration of which domain is needed for this problem
* definitions of objects belonging to each type
* definition of the initial state (list - not a formula - of predicates which are initially true)
* definition of goal states (specified as a formula, like how we define the preconditions for actions)
* `)`

In the actual syntax of PDDL, such a file might look as follows:
```
(define (problem XXX)
    (:domain XXX)  

    (:objects XXX)

    (:init (XXX))

    (:goal (XXX))
)
```
**Problem Name:** First off, we start by giving our problem a name, just as we did for the domain file, which we do in the first line of the problem file as follows: `(define (problem simple_example)`. Thus, our problem will just be called `simple_example`.

**Domain:** Next, we need to specify in which domain our problem is actually posed. You can have multiple problem files that are all based on the same domain file (in fact, we'll see this later on). In our case, we will just declare `(:domain simple)` in order to tell the PDDL solver that it should associate our problem with the domain `simple`.

**Objects:** Here we instantiate the actual things in our problem, for example if we want to create two new `location`-type objects called `Shelf` and `Table` that will represent our shelf and table, respectively, from the example we've been following. Let's instantiate the four objects that we'll need for our problem:
```
(:objects
    Apple - apple
    Robot - robot
    Shelf Table - location
)
```
Note that PDDL delimits code blocks using parentheses, and so indententation is not important as it is with Python. Thus, we can write the objects declaration line as just `(:objects Apple - apple Robot - robot Shelf Table - location)`. Do whatever is convenient for you.

Here is a tree showing the object-type hierarchy in our problem (thanks to the VS Code PDDL Extension):
<p align="center">
  <img src="Graphics/simple_objects.png" />
</p>
Here, the blue ovals are the types that we defined in the domain file, and the yellow rectangles are the object instances that we defined in the problem file.

So far, our problem file looks like this:
```
(define (problem simple_example)
        (:domain simple)
	    
        (:objects
        Apple - apple
        Robot - robot
        Shelf Table - location
        )
)
```
In fact, we're almost done with it!

**Initial State:** Next we'll want to specify our initial state, i.e. the unordered set of predicates that hold true at the start of our problem. All predicates which are not explicitly said to be true in the initial conditons are assumed by PDDL to be false (unless the domain declares the requirement `:open-world`, but we won't worry about that) [2]. So, what actually holds true at the start of our problem? Well, we know that the apple is on the shelf (and not on the table), the robot is not holding the apple, and the robot is at the table (and not at the shelf). We don't need to specify the predicates which hold false, so we shall only declare the predicates which hold true; in this case, they are `(On Apple Shelf)` and `(At Robot Table)`. Thus, we can declare the initial state for our example as follows: `(:init (On Apple Shelf)(At Robot Table))`

**Goal State:** Finally, we need to tell PDDL what we want our end state (i.e. what predicates need to hold true/false) to be. Thinking about our problem, we really just want the apple to be on the table - we don't have any other requirements for the robot. Thus, we only need one statement to hold true, `(On Apple Table)`, and so we can specify our goal state as follows: `(:goal (On Apple Table))`.

Putting it all together, we get our complete problem file:
```
(define (problem simple_example)
        (:domain simple)

	(:objects
        Apple - apple
        Robot - robot
        Shelf Table - location
        )
        
	(:init (On Apple Shelf)(At Robot Table))

	(:goal (On Apple Table))
)
```
You can find this problem as a `.pddl` file called `problem_simple.pddl` in the `Simple Example` folder. Now that we have our domain and problem files ready, we can send them to the PDDL solver, which will use some sort of AI algorithm (more on this later) to find a suitable plan - a sequence of actions that will take us from the initial state to the goal state. Running such a solver, we will obtain the following plan (for me it did so in about 1 second):
```
(move Robot Table Shelf)
(pick Apple Robot Shelf)
(move Robot Shelf Table)
(place Apple Robot Table)
```
Here is a visual representation of that plan:
<p align="center">
  <img src="Graphics/simple_plan.png" />
</p>

This is exactly the solution we humans would have proposed as well, but this was a rather simple problem. Let's look at a bit of a trickier problem (at least for us humans), and see how we can represent it in PDDL and solve it quite easily using AI planning solvers.

## (2) Ex: The Farmer's Dilemma

### (2.1) Problem Statement
A farmer lives on a small plot of land next to a river. One day, he travels across the river in a small boat and purchases a fox, a chicken, and a bag of seed from a feed and supply store. When the farmer returns to his boat to cross the river again and go home, he realizes he has a dilemma.

The farmer can only take one item in his small boat at a time, otherwise he risks capsizing. He cannot leave the fox alone with the chicken, because the fox will eat the chicken. He cannot leave the chicken alone with the seed, because the chicken with eat the seed.

How does the farmer successfully get all three items across the river? [[1]](https://www.popularmechanics.com/technology/a23689/riddle-of-the-week-1-the-farmers-dilemma/)

<p align="center">
  <img src="Graphics/farmer.jpg" />
</p>

### (2.2) First Steps & Building the Domain File
Let's try to visualize the problem and all of the objects within it that might be relevant for us:
<p align="center">
  <img src="Graphics/farmers_domain.png" />
</p>
We see that we have a rational agent in the farmer himself, three goods in the seed, chicken, and fox, a vehicle in the boat, and three locations in the left and right banks of the river as well as the river itself. Which of these will actually be relevant for defining the problem in PDDL? It shouldn't be difficult to see why we definitely need to have the fox, chicken, and seed, as well the right and left banks of the river. Keeping the boat will also serve useful. Do we really need the farmer and the river, though? It should be easy to convince ourselves that the river won't be so useful, but maybe more surprisingly we don't really need to represent the farmer (you'll just have to trust me on that for now).

So we've agreed that we need to represent the fox, chicken, seed, boat, left bank, and right bank. What kind of hierarchy might be useful here? Starting with the first three, we see that they are all goods that the farmer wants to move across the river, and it could be useful to define a new type called `good` that extens the `object` type. Then, for each of the three goods themselves we can define new subtypes of the `good` type - this will help us later on to establish the relations between them, i.e. which ones can be together at the same time - simply called `fox`, `chicken`, and `seed`. Next, we see that the left and right banks are both just locations where the goods, boat, and farmer can be at, and so it might be smart to define a new type called `location` as we did in the previous example to represent them. We won't need to create new subtypes for the right and left bank individually as we did for the goods, and it'll be more clear why that is as we mvoe forward. Lastly, we can treat the boat as its own new type called `boat`, since we will likely want to define unique predicates and actions just for it.

Thus, we start building our domain file as follows:
```
(define (domain farmer)
    (:requirements :typing :adl :universal-preconditions)  

    (:types boat location good - object
            fox chicken seed - good
    )
```
Notice we have added a new requirement called `:universal-preconditions`, we will explore it once we get to the action definitions.

The following figure can help us visualize the hierarchy of the types that we've just established:
<p align="center">
  <img src="Graphics/farmer_domain_types.png" />
</p>

Next we'll want to define our predicates. Let's think of some properties of our objects that would be useful to represent as boolean functions. An obvious question to start with is "Where is object *x*?", which can reformulate as a yes/no question: "Is object *x* at location *y*?". Framing the question this way, we see that we might want to define a predicate that checks whether an object is at a certain location. It's clear that the object in question can be any of the goods, and the location in question can be either of the banks. What about the boat? Well, we can only load a good onto the boat if the boat is at the same bank as that good. Since the boat is also of type `object` (as are all the goods) due to the hierarchy we've defined, we can define a predicate `(at ?o - object ?l - location)` to check if some object is at some location. Note that the banks are also objects, and this predicate could validly check if the right bank is at the right bank (though this won't be very helpful to us).

Now let's focus on how we'd like the boat to function in relation to the goods. Rather than redefine the boat as a subtype of the `location` type so that we can just apply the `(at ?o - object ?l - location)` predicate to check if a good is on the boat, we will define a new predicate that checks if a certain good is on the boat - let's define it as a binary predicate (we won't add more atoms since the boat can only carry one good at a time): `(on-boat ?g - good ?b - boat)`. What if there are no goods on the boat (which is a necessary condition for loading any good onto the boat)? Let's define one last unary predicate to check if the boat is clear: `(boat-clear ?b - boat)`.

To summarize, our predicate definition in the domain file should look as follows:
```
(:predicates (at ?o - object ?l - location)(on-boat ?o - object ?b - boat)(boat-clear ?b - boat))
```

With all our predicates at hand, we can start to define our actions. Let's try to come up with some actions that should exist in our world and will be relevant for our problem. Taking inspiration from our previous example, we can once again reason that there should be three such actions: one which loads a good to the boat from one of the banks, one that moves the boat between the banks (it doesn't necessarily need to have a good on it when it does so - we'll see why), and one that unloads the good from the boat to the bank where the boat is located. The first and last one are pretty straightforward, and we can define that as follows:
```
(:action load
       :parameters (?g - good ?l - location ?b - boat)
       :precondition (and (at ?g ?l)
                          (at ?b ?l)
                          (boat-clear ?b))
       :effect (and (on-boat ?g ?b)
                    (not (boat-clear ?b))
                    (not (at ?g ?l)))
)

(:action unload
       :parameters (?g - good ?l - location ?b - boat)
       :precondition (and (on-boat ?g ?b)
                          (at ?b ?l)
                          (not (boat-clear ?b)))
       :effect (and (at ?g ?l)
                    (boat-clear ?b)
                    (not (on-boat ?g ?b)))
)
```
We can see that these actions are almost mirror images of each other. For the loading action, we require that the good and boat both be at the same location, and that the boat be clear of any goods. The effect of this loading action is that the good is now on the boat and no longer at the original location, and that the boat is no longer clear. The unloading action on the other hand requires that the good be on the boat (which implies that the boat is not clear, but we will explicitly require this in the preconditions anyways) and that the boat be at the desired dropoff location. The effect of this unloading action is that the good is now at the desired location and no longer on the boat, and thus the boat is once again clear.


Next, we'll want to define the action that moves the boat from one bank to the other. Here we must remember the following constraint from the problem statement: "[The farmer] cannot leave the fox alone with the chicken, because the fox will eat the chicken. He cannot leave the chicken alone with the seed, because the chicken with eat the seed." So, the boat can't sail with a certain good to the other bank if it means that we will leave a dangerous pair (fox-chicken or chicken-seed) unsupervised at one of the banks. We can represent this constraint as a precondition for our new action, which we'll call `sail`, and we can do so as follows:
```
(forall (?l - location)
    (and (not (and (at ?f ?l)
              (at ?c ?l)))
         (not (and (at ?c ?l)
              (at ?s ?l))))
)
```
Whoa, what's this new thing called `forall`? Well if you remember back to the requirements definition, we added a new one called `:universal-preconditions`, and what this does is it allows us to use a new condition that must hold true across all objects of a type, or just all objects. In this case, we have asked that for all possible objects of type `location` (i.e. the left and right banks), that we can't permit the case where the fox and chicken are both at that location at the same time, nor the case where the chicken and seed are both at that location at the same time. It might be easier to show this in Python:

In [8]:
locations = ['left bank', 'right bank']
fox = {'name' : 'fox', 'location' : 'left bank'}
chicken = {'name' : 'chicken', 'location' : 'right bank'}
seed = {'name' : 'seed', 'location' : 'right bank'}
### Here, the chicken and seed are both on the right bank at the same time.
### Thus, we expect to obtain False from our forall() function.

def at(object,location):
    if object['location'] == location:
        return True
    else:
        return False

def forall(locations, fox, chicken, seed):
    check = []
    for l in locations:
        if not (at(fox,l) and at(chicken,l)) and not (at(chicken,l) and at(seed,l)):
            check.append(True)
        else:
            check.append(False)
    return check[0] and check[1]

print(forall(locations, fox, chicken, seed))

False


Here, the boolean function `at(object,location)` is equivalent to the predicate `(at ?o - object ?l - location)` that we defined earlier.

With this new `forall` condition at hand, we can now properly define our `sail` action:
```
(:action sail
       :parameters (?from ?to - location ?b - boat ?f - fox ?c - chicken ?s - seed)
       :precondition (and (at ?b ?from)
                          (forall (?l - location)
                              (and (not (and (at ?f ?l)
                                        (at ?c ?l)))
                                   (not (and (at ?c ?l)
                                        (at ?s ?l))))
                          )
                     )
       :effect (and (at ?b ?to)
                    (not (at ?b ?from)))
)
```
The preconditions of the action are that the boat must be at the original location, and that the dangerous pairs will not be left together at one of the banks when the boat sails away (this is the `forall` condition). The effect of this action is that the boat will now be at the desired location, and no longer at the original location.

Now that we have all of our actions defined, we can produce the domain file for the Farmer's Dilemma as follows:
```
(define (domain farmer)
    (:requirements :typing :adl :universal-preconditions)  

    (:types boat location good - object
            fox chicken seed - good
    )

    (:predicates (at ?o - object ?l - location)(on-boat ?g - good ?b - boat)(boat-clear ?b - boat) )

    (:action load
       :parameters (?g - good ?l - location ?b - boat)
       :precondition (and (at ?g ?l)
                          (at ?b ?l)
                          (boat-clear ?b))
       :effect (and (on-boat ?g ?b)
                    (not (boat-clear ?b))
                    (not (at ?g ?l)))
    )

    (:action sail
       :parameters (?from ?to - location ?b - boat ?f - fox ?c - chicken ?s - seed)
       :precondition (and (at ?b ?from)
                          (forall (?l - location)
                              (and (not (and (at ?f ?l)
                                        (at ?c ?l)))
                                   (not (and (at ?c ?l)
                                        (at ?s ?l))))
                          )
                     )
       :effect (and (at ?b ?to)
                    (not (at ?b ?from)))
    )

    (:action unload
       :parameters (?g - good ?l - location ?b - boat)
       :precondition (and (on-boat ?g ?b)
                          (at ?b ?l)
                          (not (boat-clear ?b)))
       :effect (and (at ?g ?l)
                    (boat-clear ?b)
                    (not (on-boat ?g ?b)))
    )
)
```
You can find this domain as a `.pddl` file called `domain_farmer.pddl` in the `Farmer's Dilemma` folder.

### (2.3) The Problem File

Now we need to build the problem file, and we'll start it as follows:
```
(define (problem farmers_dilemma)
        (:domain farmer)
	    
        (:objects
        Fox - fox
        Chicken - chicken
        Seed - seed
        Boat - boat
        Left_Bank Right_Bank - location
        )
```
Here we defined the problem name, we referenced the name of the domain over which the problem is defined, and we instantiated the relevant objects as we discussed earlier. The object hierarchy looks as follows:
<p align="center">
  <img src="Graphics/farmer_problem_objects.png" />
</p>

Next, we'll re-examine the original problem definition in order to discern what the initial and goal states are. We are told that the farmer starts with the fox, chicken, and seed on one side of the river (and the boat is one the same side as them), and he would like to move all of his goods across the river safely. The initial state should thus be defined as follows:
```
(:init
(at Fox Left_Bank)(at Chicken Left_Bank)(at Seed Left_Bank)
(at Boat Left_Bank)(boat-clear Boat)
)
```    
Remember, we don't need to specify the predicates which are initially False (i.e. `(at Boat Right_Bank)`), we only need to specify the ones that are initially True. Here, we declared that the fox, chicken, seed, and boat are all located at the left bank in the beginning, and that the boat is initially clear of any goods.

Finally, we can specify our goal state as follows:
```
(:goal (and (at Fox Right_Bank)
            (at Chicken Right_Bank)
            (at Seed Right_Bank)
            (at Boat Right_Bank)
        )
)
```
Here, our goal state only requires that the fox, chicken, seed, and boat all be located on the right bank at the end of the plan (we didn't really need to specify that the boat must be at the right bank at the goal state, since it's obvious that it will be there if all of the goods are there). We don't need to clarify that the boat is clear at the end, since (if we've defined our domain well) there is no conceivable way for the boat to not be clear when all of the goods are at the right bank.

Putting it all together, we obtain our problem file for the Farmer's Dilemma:
```
(define (problem farmers_dilemma)
        (:domain farmer)

	(:objects
        Fox - fox
        Chicken - chicken
        Seed - seed
        Boat - boat
        Left_Bank Right_Bank - location
        )
        
	(:init
        (at Fox Left_Bank)(at Chicken Left_Bank)(at Seed Left_Bank)
        (at Boat Left_Bank)(boat-clear Boat)
        )

	(:goal (and (at Fox Right_Bank)
                    (at Chicken Right_Bank)
                    (at Seed Right_Bank)
                    (at Boat Right_Bank)
                )
        )
)
```
You can find this problem as a `.pddl` file called `problem_farmer.pddl` in the `Farmer's Dilemma` folder.

### (2.4) The Solution

Now that we have built our domain and problem files, we can finally ask an AI planner to solver the Farmer's Dilemma for us. Doing so, we obtain a 17-step solution (so 18 states total) as follows:
```
(load chicken left_bank boat)
(sail left_bank right_bank boat fox chicken seed)
(unload chicken right_bank boat)
(sail right_bank left_bank boat fox chicken seed)
(load seed left_bank boat)
(sail left_bank right_bank boat fox chicken seed)
(unload seed right_bank boat)
(load chicken right_bank boat)
(sail right_bank left_bank boat fox chicken seed)
(unload chicken left_bank boat)
(load fox left_bank boat)
(sail left_bank right_bank boat fox chicken seed)
(unload fox right_bank boat)
(sail right_bank left_bank boat fox chicken seed)
(load chicken left_bank boat)
(sail left_bank right_bank boat fox chicken seed)
(unload chicken right_bank boat)
```

This is a lot of actions, so let's break this up into a few steps at a time and try to visualize what the plan is telling us to do:
<p align="center">
  <img src="Graphics/farm_sol_1.png" />
</p>
In the first 3 actions, the farmer loads the chicken onto the boat, sails across to the right bank, and unloads the chicken at that bank.
<p align="center">
  <img src="Graphics/farm_sol_2.png" />
</p>
In the next 8 actions, the farmer sails back to the left bank, loads the seed onto the boat, and then sails again to the right bank. He then unloads the seed onto the right bank, but we notice that this leaves us in a state (denoted 7) where both the chicken and the seed are at the right bank, and this could turn into a dangerous pair. Fret not, as the solver recognizes the constraint we set for this scenario and tells the farmer to quickly load the chicken back onto the boat. He then sails to the left bank and unloads the chicken, but we once again run into a potential dangerous pair: the fox and chicken are now both at the left bank. The farmer then quickly loads the fox onto the boat to prevent it from eating the chicken.
<p align="center">
  <img src="Graphics/farm_sol_3.png" />
</p>
In the final 6 actions, the farmer sails the fox across to the right bank, unloads it there (notice that the fox is now sitting with the seed on the right bank, which is perfectly fine), and sails back to the left bank without any goods onboard. He loads the chicken back onto the boat, sails across the river one last time, and finally unloads the chicken onto the right bank to complete the plan and solve his dilemma!

This example hopefully showed you the potential that PDDL offers, as the AI solver resolved the dilemma in less than a second, while a regular human might need a few minutes to correctly reason out a solution.

## (3) Conclusion

In this tutorial, we learned about classical planning problems and how we can go about representing them using the Planning Domain Definition Language (PDDL). We touched on points such as:

* How to define a domain file in PDDL (including requirements, types, predicates, and actions)
* How to define a problem file in PDDL (including objects, initial states, and goal states)
* How to apply PDDL solvers to basic task planning problems

Next week, we will continue to work with PDDL and model some more seemingly tricky problems (such as the Towers of Hanoi). We will discuss the state space of a classical planning problem, and some of the approaches used by different AI planners to solve PDDL-represented problems and create valid plans for them.

#### ***Credit:** Some parts of this tutorial (mainly the first half) were adapted from [Robotic Sea Bass](https://roboticseabass.com/2022/07/19/task-planning-in-robotics/)'s fantastic article on task planning in robotics. The tutorial was written by Yotam Granov.*

### **References**
[1] Fikes & Nilsson. ["STRIPS: A New Approach to the Application of Theorem Proving to Problem Solving"](https://ai.stanford.edu/users/nilsson/OnlinePubs-Nils/PublishedPapers/strips.pdf), 1971.

[2] Ghallab et. al. ["PDDL - The Planning Domain Definition Language"](https://www.researchgate.net/publication/2278933_PDDL_-_The_Planning_Domain_Definition_Language), 1998.

[3] Srivastava et. al. ["A Modular Approach to Task and Motion Planning with an Extensible Planner-Independent Interface Layer"](https://people.eecs.berkeley.edu/~russell/papers/icra14-planrob.pdf), 2013.

[4] "Task Planning in Robotics" article by Robotic Sea Bass: https://roboticseabass.com/2022/07/19/task-planning-in-robotics/

[5] Dr. Alex Shleyfman's Technion "Cognitive Robotics" course tutorials, 2022. (You can find these on the course Moodle, under the 'Old Tutorials' section)